In [26]:
import gpxpy
import psycopg2

In [27]:
filename = '11398719881.gpx'

In [28]:
with open('../data/'+ filename, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

In [46]:
gpx_points = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            timestamp = point.time.strftime('%Y-%m-%d %H:%M:%S')
            gpx_points.append((timestamp, point.latitude, point.longitude))

In [47]:
conn = psycopg2.connect(
    host="localhost",
    database="gpx",
    user="root",
    password="root"
)

In [48]:
cursor = conn.cursor()

In [54]:
create_table_query = """
DROP TABLE IF EXISTS gpx_points;

CREATE TABLE IF NOT EXISTS gpx_points (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP,
    filename TEXT,
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    location GEOGRAPHY(POINT, 4326)
);
"""
cursor.execute(create_table_query)

In [55]:
try:
    for point in gpx_points:
        time, latitude, longitude = point
        cursor.execute("""
            INSERT INTO gpx_points (timestamp, filename, latitude, longitude, location)
            VALUES (%s, %s, %s, %s, ST_SetSRID(ST_MakePoint(%s, %s), 4326))
        """, (time, filename, latitude, longitude, longitude, latitude))      
    conn.commit()
except Exception as e:
    conn.rollback()